In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve as ngs
import NgsAMG as amg

## HDiv AMG

Non-standard monolithic AMG method for Stokes, works by coarsening dual graph and constructing coarse basis functions with piecewise constant divergence.

Similar to [ParELAG](https://github.com/LLNL/parelag), see also "Algebraic multigrid for k‐form Laplacians" (Luke Olson, Nathan Bell).


In [15]:
from usrMtgStuff import GetValve, StokesHDGDiscretization, solveCondensed

mesh = GetValve(N=4, dim=2, maxh=0.1, closevalve=False)
uin = CF((1, 0)) if mesh.ngmesh.dim == 2 else CF((1,0,0))

wall = "wall" if mesh.ngmesh.dim == 3 else "default"
(V, a, f, u) = StokesHDGDiscretization(mesh, order=2, wall=wall, inlet="inlet", outlet="outlet", nu=1e-3, div_div_pen=1e6)


In [16]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [17]:

amg_cl = amg.stokes_hdiv_gg_2d if V.mesh.ngmesh.dim == 2 else amg.stokes_hdiv_gg_3d

c = amg_cl(a, ngs_amg_pres_vecs="P1", ngs_amg_sm_type="hiptmair", ngs_amg_sm_type_range="dyn_block_gs")

f.Assemble()
a.Assemble()

u.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
solveCondensed(a, c, u.vec, f.vec, tol=1e-8)


Set up AMG-levels...
 map maps 49039 -> 25116, fac 0.512164
 map maps 25116 -> 6547, fac 0.26067
 map maps 6547 -> 896, fac 0.136857
 map maps 896 -> 103, fac 0.114955
 map maps 103 -> 29, fac 0.281553
Done setting up AMG-levels!

 ---------- AMG Summary ---------- 
Vertex complexity: 1.46579
Operator complexity: 1.87965
Vertex complexity components: 1 0.368355 0.0808005 0.0134459 0.00247029 0.000719199 
Operator complexity components: 1 0.544199 0.29092 0.0407167 0.0030033 0.000808682 
# vertices 31980 11780 2584 430 79 23 
# edges: 49039 25116 6547 896 103 29 
# procs: 1 1 1 1 1 1 
NZEs:6014725 3273208 1749804 244900 18064 4864 
 ---------- AMG Summary End ---------- 

Preconditioner test:
   min EV = 0.05457451753302116
   max EV = 0.999890266402498
   condition = 18.32155943105679


Solve...
CG iteration 1, residual = 150.81465700448615     
CG iteration 2, residual = 13.102371507751082     
CG iteration 3, residual = 2.0918493952147013     
CG iteration 4, residual = 0.30029794433

In [18]:
Draw(u.components[0], vectors=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [19]:
for t in Timers():
    if t["time"] > 0.05:
      print(f"{t['name']}: {t['time']}")

AMG - Solve: 1.9179291164897563
Finalize tmp timer: 0.07856142555837405
AMG - Solve: 0.13006918620158078
DynBlockSmoother - SmoothRHS: 1.4043153431916129
HiptMairSmoother::SmoothBack - range: 1.4045482721130063
HiptMairSmoother::SmoothBack: 1.9543041702628299
DynBlockSparseMatrix::Mult: 0.3222207619757672
DynBlockSmoother - SmoothRes: 0.8348869648970924
DynBlockSparseMatrix::MultAdd: 0.9841364263547534
HiptMairSmoother::ApplyPotUpdate: 0.5565781065246358
GSS3<bs=1>::SmoothRHS: 0.06076286913939977
HiptMairSmoother::CalcPotRhs: 0.4848561749557195
HiptMairSmoother::Smooth - range: 0.8351922452265595
HiptMairSmoother::Smooth: 1.3917519373025395
level 2: 0.40357421366739954
level 1: 0.9037330957946826
level 0: 2.566449657382557
AMGMatrix::Mult: 3.9179111416336276
SparseMatrix::MultAdd: 0.6524635876018023
DynBlockSparseMatrix - Finalize: 0.2247670563762835
StokesAMGFactory::ProjectToPotSpace: 0.10298112110773665
StokesAMGFactory::MapLevel: 2.3897406109019332
StokesAMGFactory::BuildPWProl_imp

In [20]:
bf = GridFunction(V)
# Draw(bf.components[0], vectors=True)
# Draw(div(bf.components[0]), mesh)

@interact(level=widgets.IntSlider(min=0, max=c.GetNLevels()-1, step=1, value=c.GetNLevels()-1, continuous_update=False), \
          dof=widgets.IntSlider(min=0, max=30, step=1, value=0, continuous_update=False),\
          drawDiv=True)
def getDrawBF(level, dof, drawDiv):
    c.GetBF(vec=bf.vec, level=level, dof=dof, comp=0)
    if drawDiv:
        Draw(div(bf.components[0]), mesh)
    else:
        Draw(bf.components[0], vectors=True)
    # Redraw()

# interact(lambda level, dof: getDrawBF(level, dof), level=c.GetNLevels()-1, dof=widgets.IntSlider(min=0, max=500, step=1, value=0));

interactive(children=(IntSlider(value=5, continuous_update=False, description='level', max=5), IntSlider(value…

In [21]:
c.GetLoop(comp_vec=bf.vec, level=3, loop_num=1)



In [26]:
bf = GridFunction(V)
# Draw(bf.components[0], vectors=True)
# Draw(div(bf.components[0]), mesh)

@interact(level=widgets.IntSlider(min=0, max=c.GetNLevels()-2, step=1, value=c.GetNLevels()-2, continuous_update=False), \
          loop=widgets.IntSlider(min=0, max=500, step=1, value=107, continuous_update=False),\
          drawDiv=False)
def getDrawLoop(level, loop, drawDiv):
    c.GetLoop(comp_vec=bf.vec, level=level, loop_num=loop)
    if drawDiv:
        Draw(div(bf.components[0]), mesh)
    else:
        Draw(bf.components[0], vectors=True)
    # Redraw()



interactive(children=(IntSlider(value=4, continuous_update=False, description='level', max=4), IntSlider(value…